# Import Libraries

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Set up the Parameters of the Message

Navigate to [App Password Generator](https://accounts.google.com/v3/signin/identifier?continue=https%3A%2F%2Fmyaccount.google.com%2Fapppasswords&followup=https%3A%2F%2Fmyaccount.google.com%2Fapppasswords&ifkv=ARZ0qKK7s2CXFItpn5kqUfS9HX2bc7oqnphu9lJKhLkdDTNv8ZmIlrf2_2ow7qS0rc1xq4m_o2f9TA&osid=1&passive=1209600&rart=ANgoxceHvwqJVyJsrFAnBM-GTNddR6_rT-3w_YVwx9tLMf2v8J27hjvo0tjI2nHIlfxtowoLqF6D17T29rUzibwg-vsUf8ygLD9EN5gCD2XWRiJo0F4YQWg&service=accountsettings&flowName=GlifWebSignIn&flowEntry=ServiceLogin&dsh=S1158263025%3A1711295381941536&theme=mn&ddm=0), designate an app name such as "security project," and obtain a 16-digit password. Copy this password and paste it into the designated password field as instructed.

In [ ]:
password = ""
from_email = ""  # must match the email used to generate the password
to_email = ""  # receiver email

# Server Creation and Authentication

In [ ]:
server = smtplib.SMTP('smtp.gmail.com: 587')
server.starttls()
server.login(from_email, password)

# Email Send Function

In [ ]:
def send_email(to_email, from_email):
    message = MIMEMultipart()
    message['From'] = from_email
    message['To'] = to_email
    message['Subject'] = "Security Alert"
    # Add in the message body
    message_body = f"ALERT - Someone is moving into the restricted area!"

    message.attach(MIMEText(message_body, "plain"))
    server.sendmail(from_email, to_email, message.as_string())

# Object Detection and Alert Sender

In [ ]:
class ObjectDetection:
    def __init__(self, capture_index):
        # default parameters
        self.capture_index = capture_index
        self.email_sent = False

        # model information
        self.model = YOLO("yolov8n.pt")

        # visual information
        self.annotator = None
        self.start_time = 0
        self.end_time = 0

        # device information
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

    def predict(self, im0):
        results = self.model(im0)
        return results

    def display_fps(self, im0):
        self.end_time = time()
        fps = 1 / np.round(self.end_time - self.start_time, 2)
        text = f"FPS: {int(fps)}"
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1.0, 2)[0]
        gap = 10
        cv2.rectangle(im0, (20 - gap, 70 - text_size[1] - gap), (20 + text_size[0] + gap, 70 + gap), (255, 255, 255), -1)
        cv2.putText(im0, text, (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)

    def plot_bboxes(self, results, im0):
        flag = False
        self.annotator = Annotator(im0, example=results[0].names)
        boxes = results[0].boxes.xyxy.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        names = results[0].names
        for box, cls in zip(boxes, clss):
            if cls == 0:    # Person class
                self.annotator.box_label(box, label=names[int(cls)], color=colors(int(cls), True))
                flag = True
        return im0, flag

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        while True:
            self.start_time = time()
            ret, im0 = cap.read()
            assert ret
            results = self.predict(im0)
            im0, flag = self.plot_bboxes(results, im0)

            if flag:  
                if not self.email_sent:    # Only send email if not sent before
                    send_email(to_email, from_email)
                    self.email_sent = True
            else:
                self.email_sent = False

            self.display_fps(im0)
            cv2.imshow("YOLOv8 Detection", im0)
            if cv2.waitKey(5) & 0xFF == 27:    # 'Esc' key
                break
        cap.release()
        cv2.destroyAllWindows()
        server.quit()

# Call the Object Detection Class and Run the Inference

In [ ]:
detector = ObjectDetection(capture_index=0)
detector()